# Derivative. Optimization.

## Optimization

Optimization is the selection of a best element, based on some criteria, from some set of available alternatives. 

Firstly, we can categorize optimization problem in two groups according to type of the set of available values. We can optimize on:
1. Finite sets of values. For example, we want to optimize a function which will pick the best move in tic tac toe. 
2. Infinite sets of values. For example, we want find the best angle for throwing ball in a basket. <br>
These categories are also called *discrete* and *continuous* optimizations, respectively. 

Secondly, the optimization problem can be categorized by presence of randomness in criteria or set:
1. *Deterministic* optimization, which means that we always know what are we optimizing with 100 percent certainty. For example, we want to optimize a policy for playing chess, which doesn't involve any randomness
2. *Stohastic* optimization, which means that we know parametres only within certain bounds. However, we can estimate probability distribution of certain situations. For example, in contrast to deterministic chess, we want to play better at poker, where chances of victory depend on how cards in deck were shuffled. 

## Optimization methods

### Travelling salesman problem

For several future lessons we will learn more about different optimization methods and algorithms on example of travelling salesman problem. This problem asks one simple question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?" <br>
![source:Wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/GLPK_solution_of_a_travelling_salesman_problem.svg/240px-GLPK_solution_of_a_travelling_salesman_problem.svg.png)<br>
We have finite number of cities, and, obviously, finite number of different routes, which means that by definition this problem is **discrete optimization problem**. That is why we will start from discrete optimization algorithms. <br>
But before that, we will randomly define coordinates of cities and draw map of cities:

In [ ]:
def create_map(n,seed):
    import matplotlib.pyplot as plt
    import random
    random.seed(seed) #seed of random number generator, it allows to get the same map everytime

    #n is a number of cities
    x=[] #list of x coordinates of cities (in km)
    y=[] #list of y coordinates of cities (in km)
    names=[] #list of cities' names

    #now we create loop to append coordinates to lists
    for i in range(n):
        x.append(random.randint(0,1000))
        y.append(random.randint(0,1000))
        names.append(chr(65+i)) #we name cities as A,B,C...


    #finally, we plot map
    plt.figure(figsize=(7,7))
    plt.scatter(x,y)
    for i in range(n): plt.text(x[i]+10,y[i]-20,names[i]) #adding names of cities
    plt.show()
    return x,y,names #returns coordinates of cities

In [ ]:
x,y,names=create_map(10,78)

### Discrete optimization methods

#### Brute force

Obviously, if set of available values is finite, we can loop through every possible option in finite time. Let's create dictionary, which will include each city as a key and new dictionary which includes distances to other cities from key one.

In [ ]:
def distance_dictionary(x,y):  
    n=len(x) #number of cities
    cities={} #dictionary which stores all possible distances between two cities
    for i in range(n):
        city={} #we create small dictionary which includes distances from city i to all other cities
        for k in range(n):
            if (i!=k): 
                city[k]=((x[i]-x[k])**2+(y[i]-y[k])**2)**(1/2) #calculating distance

        cities[i]=city

    return cities

In [ ]:
#Source link: https://gist.github.com/westphahl/432876
def find_paths(node, cities, path, distance): 
    #node is number of city (starting from 0)
    #cities is newly created dictionary
    #path is list of nodes which are already at a path
    #distance is complete length of a path
    
    # Add way point
    path.append(node)

    # Calculate path length from current to last node
    if len(path) > 1:
        distance += cities[path[-2]][node]

    # If path contains all cities and is not a dead end,
    # add path from last to first city and return.
    if (len(cities) == len(path)) and (path[0] in cities[path[-1]]):
        global routes
        path.append(path[0])
        distance += cities[path[-2]][path[0]]        
        routes.append([distance, path])
        return

    # Fork paths for all possible cities not yet used
    for city in cities:
        if (city not in path) and (node in cities[city]):
            find_paths(city, dict(cities), list(path), distance) 
            #if path is not complete, we recursively append all possible cities and go through every possible path         


Now, we can calculate the best route and plot it on the map.

In [ ]:
def brute_force(cities,names):
    global routes
    n=len(cities) #number of cities    
    find_paths(0, cities, [], 0) #recursive function
    routes.sort() #sort all possible routes by distances to get the best one as element number 0
    if len(routes) != 0:  
        s="" #create string for storing sequence of cities' names
        for i in range(n): s+=names[routes[0][1][i]]+"-" #add names 
        print ("Shortest route:",s[:-1])
    else:
        print ("FAIL!")    
    

In [ ]:
#After we have found the shortes path, we can show it on map
def path_on_map(routes,names):
    import matplotlib.pyplot as plt
    n=len(names) #number of cities
    optimal_x=[] #consecutive coordinates of cities from the best route
    optimal_y=[]
    for i in range(n): 
        optimal_x.append(x[routes[0][1][i]])
        optimal_y.append(y[routes[0][1][i]])

    optimal_x.append(optimal_x[0])    
    optimal_y.append(optimal_y[0])  

    plt.figure(figsize=(7,7))    
    plt.plot(optimal_x,optimal_y,'r')
    plt.scatter(x,y)
    for i in range(n): plt.text(x[i]+10,y[i]-20,names[i]) #adding names of cities
    plt.title("Distance: "+str(int(routes[0][0]))+" km")
    plt.show()

In [ ]:
routes=[] #list where we will store complete routes - routes that start and end at one city
cities=distance_dictionary(x,y)
brute_force(cities,names)
path_on_map(routes,names)

Personally for me, when I saw that path, I believed that path G-E-H-F is better than G-H-E-F and decided to check whether algorithm properly works. I printed distances between these cities and was astonished that code was right and it was just an illusion. 

In [ ]:
print("G-H =",cities[6][7],"H-E =",cities[7][4],"E-F =",cities[4][5],"G-E-H-F =",cities[6][7]+cities[7][4]+cities[4][5])
print("G-E =",cities[6][4],"E-H =",cities[4][7],"H-F =",cities[7][5],"G-H-E-F =",cities[6][4]+cities[4][7]+cities[7][5])

Indeed, path that was choosed by computer is shorter than I would probably draw.

Since we loop through every possible option, we must find the best one. However, for that particular problem as well as for other ones, brute force solution requires a lot of computational power with growth of system's syze. For instance, if we try to find the shortest path by brute force, we have to go through $(n-1)!$ different paths.<br>
For n=10 program calculates $N\approx1.8\cdot10^6$ paths, which takes several seconds, but for n=20 this number increases to $N\approx1.2\cdot 10^{18}$, which is absurdly enormous number. Program would calculate several tens of thousands years (!) to solve problem for just 20 cities. <br>
Such rapid increase is the problem for brute force algorithm at practically every instance. Therefore, this algorithm works only for very small numbers of system's parametres.

#### Greedy algorithm

First of all, we need to create a map of cities with the same function as in brute force.

In [ ]:
x,y,names=create_map(25,78)

For faster performance, we will precalculate distances between all cities

In [ ]:
routes=[] #list where we will store complete routes - routes that start and end at one city
cities={} #dictionary which stores all possible distances between two cities
for i in range(n):
    city={} #we create small dictionary which includes distances from city i to all other cities
    for k in range(n):
        if (i!=k): 
            city[k]=((x[i]-x[k])**2+(y[i]-y[k])**2)**(1/2) #calculating distance
            
    cities[i]=city  


Now, we can get *approximately optimal* route by picking the closest city from remaining ones. In attempt of improving the result, we are going to start **n** routes from **n** different cities.

In [ ]:
def greedy_algorithm(cities,names):
    routes=[]
    n=len(cities) #number of cities
    for i in range(n):
        distance=0 #start travel
        path=[] #array with consecutive numbers of cities
        path.append(i)
        for k in range(n-1):
            min_distance=2000 #we need to have initial minimal distance to the next city which we will improve
            min_j=0        
            for j in range(n):            
                if (not(j in path) and (cities[path[-1]][j]<min_distance)): #if city is closer that previous ones
                    min_distance=cities[path[-1]][j]                        #and is not in list of previous cities
                    min_j=j                                                 #than assign this distance as minimal and remember this city

            path.append(min_j)
            distance+=min_distance #travel between previous city and the closest to it

        distance+=cities[path[-1]][path[0]] #return to the initial city

        routes.append([distance,path])

    routes.sort() #sort routes by distances
    s="Shortest greedy route: "
    for i in range(n): s+=names[routes[0][1][i]]+"-"
    print(s[:-1])
    return routes

In [ ]:
cities=distance_dictionary(x,y)
routes=greedy_algorithm(cities,names)
path_on_map(routes,names)

As you can see this algorithm usually does not find the most shortest path, especially for bigger number of parameters. This behaviour is caused by inability of greedy algorithm to predict far future - it just follows the highest immediate reward, while sometimes you need to sacrifice something to get better results overall. <br>
However, this algorithm is quite fast (speed of program is proportional to $n^3$) and that is why it is used to get something close to optimal configuration and then try to improve it with different algorithms which we will talk about in the next lesson.

## Practice task

Using brute force, which is guaranteed to find the best result, and greedy algorithm functions that were used in this Jupyter notebook calculate the probability of the optimal outcome using greedy algorithm for each number of cities in range $3\leqslant n\leqslant10$<br>
**Hint 1** Probability of optimal outcome can be estimated as $$\frac{N_{opt}}{N_{all}}$$where $N_{opt}$ is number of maps where greedy algorithm produces optimal result and $N_{all}$ is number of all maps you have looped trough.<br>
**Hint 2** If you want to create random maps, you have to change seeds randomly or just use:
```python
x,y,names=create_map(number_of_cities,None)
```
**Warning** You have to consider number $N_{all}$ for big numbers of cities because whole operation can be time consuming. However, if number of all maps is quite small then probability will not be calculated with enough precision. That is why it is reasonable to calculate time of execution for small $N_{all}$ (for example 10) and then calculate maximum $N_{all}$ you can afford (*10 minutes on each number of cities will do*). And remember that later time will be an important resource. Moreover, programs can run even whole day for successful completion of a task.

Change function **path_on_map(routes,names)** to draw **the longest** route and calculate the worst path for random map using brute force (n=10).

Do you need to change **greedy_algorithm(cities, names)** to get **the longest** or near it routes if you have **already** modified **path_on_map**? If yes then modify and calculate the worst path for the same random map you have used in a previous task.